## PrepocessingData

In [41]:
# Data Preprocessing
# Notebook ini berisi tahap preprocessing data untuk dataset Iris berdasarkan hasil analisis dari DataUnderstanding notebook, termasuk penanganan outliers yang terdeteksi oleh multi-model PyCaret (ABOD, KNN, COF).

print("DATA PREPROCESSING NOTEBOOK")
print("=" * 50)
print("Tahap preprocessing data untuk dataset Iris")
print("Berdasarkan hasil analisis dari DataUnderstanding notebook")
print("Penanganan outliers dari multi-model PyCaret (ABOD, KNN, COF)")

DATA PREPROCESSING NOTEBOOK
Tahap preprocessing data untuk dataset Iris
Berdasarkan hasil analisis dari DataUnderstanding notebook
Penanganan outliers dari multi-model PyCaret (ABOD, KNN, COF)


## 1. Import Libraries dan Setup Environment

In [56]:
# Import libraries yang diperlukan untuk preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Import PyCaret untuk preprocessing dan modeling
try:
    from pycaret.datasets import get_data
    from pycaret.classification import *
    from pycaret.anomaly import *
    print("PyCaret berhasil diimport")
except (ImportError, RuntimeError) as e:
    print("PyCaret tidak tersedia atau tidak kompatibel dengan versi Python Anda.")
    print("Pesan error:", e)
    print("PyCaret hanya mendukung Python 3.9, 3.10, 3.11. Silakan gunakan versi Python yang sesuai atau lanjutkan tanpa PyCaret.")

# Atur style untuk visualisasi
plt.style.use('default')
sns.set_palette("husl")

print("Libraries berhasil diimport untuk preprocessing")
print("Libraries yang tersedia:")
print("   • Pandas & NumPy: Data manipulation")
print("   • Matplotlib & Seaborn: Visualisasi")
print("   • Scikit-learn: Preprocessing tools")
print("   • PyCaret: Advanced ML preprocessing")

# === LOAD DATASET IRIS ===
print("\n=== LOADING DATA DAN HASIL OUTLIER DETECTION ===")

try:
    # Load dataset IRIS.csv (pakai koma sebagai separator)
    IRIS = pd.read_csv("IRIS.csv")


    # Rename kolom supaya konsisten dengan sklearn
    df = df.rename(columns={
        'sepal_length': 'sepal length (cm)',
        'sepal_width': 'sepal width (cm)',
        'petal_length': 'petal length (cm)',
        'petal_width': 'petal width (cm)',
        'species': 'Class'
    })

    # Tambah kolom species numerik & species name
    df['species'] = df['Class'].map({
        'Iris-setosa': 0,
        'Iris-versicolor': 1,
        'Iris-virginica': 2
    })
    df['species_name'] = df['Class'].map({
        'Iris-setosa': 'setosa',
        'Iris-versicolor': 'versicolor',
        'Iris-virginica': 'virginica'
    })

    # Hapus kolom 'Class' biar dataset bersih
    df = df.drop('Class', axis=1)

    # Info dataset
    features = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
    print(f"\nInfo Dataset:")
    print(f"   • Ukuran: {df.shape[0]} baris, {df.shape[1]} kolom")
    print(f"   • Features: {features}")
    print(f"   • Target: species (0=setosa, 1=versicolor, 2=virginica)")

    print("\nSample Data:")
    print(df.head())

except Exception as e:
    print(f"Error loading data: {e}")
    df = None


PyCaret tidak tersedia atau tidak kompatibel dengan versi Python Anda.
Pesan error: ('Pycaret only supports python 3.9, 3.10, 3.11. Your actual Python version: ', sys.version_info(major=3, minor=12, micro=1, releaselevel='final', serial=0), 'Please DOWNGRADE your Python version.')
PyCaret hanya mendukung Python 3.9, 3.10, 3.11. Silakan gunakan versi Python yang sesuai atau lanjutkan tanpa PyCaret.
Libraries berhasil diimport untuk preprocessing
Libraries yang tersedia:
   • Pandas & NumPy: Data manipulation
   • Matplotlib & Seaborn: Visualisasi
   • Scikit-learn: Preprocessing tools
   • PyCaret: Advanced ML preprocessing

=== LOADING DATA DAN HASIL OUTLIER DETECTION ===
Error loading data: 'NoneType' object has no attribute 'rename'


## 2. Load Data dan Hasil Outlier Detection

In [49]:
# Load dataset Iris dengan hasil outlier detection dari DataUnderstanding
print("=== LOADING DATA DAN HASIL OUTLIER DETECTION ===")

try:
    # Load data dari file CSV atau PyCaret
    try:
        # Coba load dari file lokal
        df = pd.read_csv('IRIS.csv', delimiter=';')
        
        # Konversi kolom numerik yang menggunakan koma sebagai decimal separator
        numeric_columns = ['sepal length', 'sepal width', 'petal length', 'petal width']
        
        for col in numeric_columns:
            if col in df.columns:
                # Konversi koma ke titik untuk decimal
                df[col] = df[col].astype(str).str.replace(',', '.').astype(float)
        
        # Buat kolom species numerik dan species name
        df['species'] = df['Class'].map({'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2})
        df['species_name'] = df['Class'].map({'Iris-setosa': 'setosa', 'Iris-versicolor': 'versicolor', 'Iris-virginica': 'virginica'})
        
        # Rename kolom untuk konsistensi dengan format sklearn
        df = df.rename(columns={
            'sepal length': 'sepal length (cm)',
            'sepal width': 'sepal width (cm)', 
            'petal length': 'petal length (cm)',
            'petal width': 'petal width (cm)'
        })
        
        # Drop kolom yang tidak diperlukan
        if 'id' in df.columns:
            df = df.drop('id', axis=1)
        if 'Class' in df.columns:
            df = df.drop('Class', axis=1)
        
        print("Dataset Iris berhasil dimuat dari data_iris.csv")
        
    except FileNotFoundError:
        # Jika file tidak ditemukan, gunakan dataset Iris dari PyCaret
        print("File lokal tidak ditemukan. Menggunakan dataset Iris dari PyCaret...")
        try:
            df = get_data('iris')
            df['species'] = df['species'].map({'setosa': 0, 'versicolor': 1, 'virginica': 2})
            df['species_name'] = df['species'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})
            print("Dataset Iris berhasil dimuat dari PyCaret")
        except:
            print("Error: Tidak dapat memuat dataset dari PyCaret")
    
    # Define feature columns
    features = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
    
    print(f"\nInfo Dataset:")
    print(f"   • Ukuran: {df.shape[0]} baris, {df.shape[1]} kolom")
    print(f"   • Features: {features}")
    print(f"   • Target: species (0=setosa, 1=versicolor, 2=virginica)")
    
    # Tampilkan sample data
    print(f"\nSample Data:")
    print(df.head())
    
except Exception as e:
    print(f"Error loading data: {e}")
    df = None

=== LOADING DATA DAN HASIL OUTLIER DETECTION ===
Error loading data: 'Class'


In [44]:
# Outlier Detection menggunakan PyCaret
print("=== DETEKSI OUTLIER DENGAN PYCARET ===")

if df is not None:
    print("Menggunakan 3 model: ABOD, KNN, COF")
    
    try:
        # Setup PyCaret anomaly detection
        from pycaret.anomaly import setup as anomaly_setup, create_model, assign_model
        
        anomaly_env = anomaly_setup(
            data=df[features],
            session_id=42,
            train_size=1.0,
            silent=True,
            verbose=False
        )
        
        print("✓ PyCaret setup berhasil")
        
        # Jalankan 3 model outlier detection
        outlier_results = {}
        models = ['abod', 'knn', 'cof']
        
        for model_name in models:
            try:
                model = create_model(model_name, fraction=0.1)
                results = assign_model(model)
                outliers = results[results['Anomaly'] == 1].index.tolist()
                outlier_results[model_name] = outliers
                df[f'{model_name}_outlier'] = results['Anomaly']
                
                print(f"✓ {model_name.upper()}: {len(outliers)} outliers")
                
            except Exception as e:
                print(f"✗ {model_name.upper()}: error")
                outlier_results[model_name] = []
                df[f'{model_name}_outlier'] = 0
        
        # Consensus analysis
        df['consensus_score'] = df['abod_outlier'] + df['knn_outlier'] + df['cof_outlier']
        df['strong_consensus'] = (df['consensus_score'] >= 2).astype(int)
        
        strong_count = df['strong_consensus'].sum()
        print(f"\n📊 Consensus outliers (≥2 model): {strong_count}")
        
        if strong_count > 0:
            strong_indices = df[df['strong_consensus'] == 1].index.tolist()
            print(f"   Indices: {strong_indices}")
        
    except Exception as e:
        print(f"Error PyCaret: {e}")
        print("Menggunakan metode statistik...")
        
        # Fallback sederhana
        outlier_results = {'abod': [], 'knn': [], 'cof': []}
        
        for i, feature in enumerate(features):
            Q1 = df[feature].quantile(0.25)
            Q3 = df[feature].quantile(0.75)
            IQR = Q3 - Q1
            outliers = df[(df[feature] < Q1 - 1.5*IQR) | (df[feature] > Q3 + 1.5*IQR)].index.tolist()
            
            model_name = ['abod', 'knn', 'cof'][i % 3]
            outlier_results[model_name].extend(outliers[:3])
        
        for model_name, outliers in outlier_results.items():
            df[f'{model_name}_outlier'] = 0
            if outliers:
                df.loc[outliers, f'{model_name}_outlier'] = 1
        
        df['consensus_score'] = df['abod_outlier'] + df['knn_outlier'] + df['cof_outlier']
        df['strong_consensus'] = (df['consensus_score'] >= 2).astype(int)
        
        print("✓ Statistik outlier detection selesai")
        
else:
    print("Data tidak tersedia")

=== DETEKSI OUTLIER DENGAN PYCARET ===
Data tidak tersedia


## 3. Outlier Removal dan Imputation dengan Visualisasi

In [45]:
# Outlier Removal dan Imputation Process
print("=== OUTLIER REMOVAL DAN IMPUTATION PROCESS ===")

if df is not None and 'consensus_score' in df.columns:
    
    # Identifikasi outliers berdasarkan strong consensus (≥2 models agree)
    outlier_mask = df['strong_consensus'] == 1
    outlier_indices = df[outlier_mask].index.tolist()
    outlier_count = len(outlier_indices)
    
    print(f"\nOutliers teridentifikasi (Strong Consensus):")
    print(f"   • Jumlah: {outlier_count} dari {len(df)} data ({outlier_count/len(df)*100:.1f}%)")
    print(f"   • Indices: {outlier_indices}")
    
    # STEP 1: Visualisasi Koordinat Data Original
    print(f"\nSTEP 1: Visualisasi Koordinat Data Original")
    
    # Tampilkan koordinat outliers dan normal data
    print(f"\nKoordinat Outliers (Strong Consensus):")
    print(f"{'Index':<6} {'Sepal_L':<8} {'Sepal_W':<8} {'Petal_L':<8} {'Petal_W':<8} {'Species':<8}")
    print("-" * 55)
    
    outlier_coords = df[outlier_mask]
    for idx in outlier_indices:
        row = df.loc[idx]
        species_name = ['setosa', 'versicolor', 'virginica'][int(row['species'])]
        print(f"{idx:<6} {row[features[0]]:<8.2f} {row[features[1]]:<8.2f} {row[features[2]]:<8.2f} {row[features[3]]:<8.2f} {species_name:<8}")
    
    # Visualisasi scatter plot koordinat 2D
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    fig.suptitle('Koordinat Data Original - Normal vs Outliers', fontsize=16, fontweight='bold')
    
    # Plot 1: Sepal Length vs Sepal Width
    normal_data = df[~outlier_mask]
    outlier_data = df[outlier_mask]
    
    # Normal data
    axes[0].scatter(normal_data[features[0]], normal_data[features[1]], 
                   c=normal_data['species'], cmap='viridis', alpha=0.7, s=50, label='Normal')
    
    # Outlier data
    if len(outlier_data) > 0:
        axes[0].scatter(outlier_data[features[0]], outlier_data[features[1]], 
                       color='red', s=100, marker='x', label=f'Outliers ({len(outlier_data)})')
    
    axes[0].set_xlabel('Sepal Length (cm)')
    axes[0].set_ylabel('Sepal Width (cm)')
    axes[0].set_title('Sepal Coordinates')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot 2: Petal Length vs Petal Width
    axes[1].scatter(normal_data[features[2]], normal_data[features[3]], 
                   c=normal_data['species'], cmap='viridis', alpha=0.7, s=50, label='Normal')
    
    if len(outlier_data) > 0:
        axes[1].scatter(outlier_data[features[2]], outlier_data[features[3]], 
                       color='red', s=100, marker='x', label=f'Outliers ({len(outlier_data)})')
    
    axes[1].set_xlabel('Petal Length (cm)')
    axes[1].set_ylabel('Petal Width (cm)')
    axes[1].set_title('Petal Coordinates')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # STEP 2: Remove Outliers (Set as NaN)
    print(f"\nSTEP 2: Menghapus Outliers (Set sebagai NaN)")
    
    # Buat copy dataset dan set outliers sebagai NaN
    df_with_nan = df.copy()
    
    for feature in features:
        df_with_nan.loc[outlier_mask, feature] = np.nan
    
    # Cek jumlah missing values
    missing_count = df_with_nan[features].isnull().sum()
    print(f"\nMissing values setelah outlier removal:")
    for feature in features:
        missing = missing_count[feature]
        pct = (missing / len(df_with_nan)) * 100
        print(f"   • {feature}: {missing} missing ({pct:.1f}%)")
    
    # STEP 3: Koordinat Data dengan Missing Values
    print(f"\nSTEP 3: Koordinat Data dengan Missing Values")
    
    print(f"\nKoordinat Data Setelah Outlier Removal (NaN untuk outliers):")
    print(f"{'Index':<6} {'Sepal_L':<8} {'Sepal_W':<8} {'Petal_L':<8} {'Petal_W':<8} {'Status':<8}")
    print("-" * 55)
    
    # Show sample missing data coordinates
    missing_indices = df_with_nan[df_with_nan[features].isnull().any(axis=1)].index.tolist()
    for idx in missing_indices[:5]:  # Show first 5 missing
        row = df_with_nan.loc[idx]
        print(f"{idx:<6} {'NaN':<8} {'NaN':<8} {'NaN':<8} {'NaN':<8} {'Missing':<8}")
    
    # Visualisasi koordinat data yang tersisa
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    fig.suptitle('Koordinat Data Setelah Outlier Removal', fontsize=16, fontweight='bold')
    
    # Data yang masih tersedia (non-NaN)
    available_mask = ~df_with_nan[features].isnull().any(axis=1)
    available_data = df_with_nan[available_mask]
    
    # Plot 1: Sepal coordinates
    axes[0].scatter(available_data[features[0]], available_data[features[1]], 
                   c=available_data['species'], cmap='viridis', alpha=0.7, s=50)
    axes[0].set_xlabel('Sepal Length (cm)')
    axes[0].set_ylabel('Sepal Width (cm)')
    axes[0].set_title(f'Sepal Coordinates\n({len(available_data)} available points)')
    axes[0].grid(True, alpha=0.3)
    
    # Plot 2: Petal coordinates
    axes[1].scatter(available_data[features[2]], available_data[features[3]], 
                   c=available_data['species'], cmap='viridis', alpha=0.7, s=50)
    axes[1].set_xlabel('Petal Length (cm)')
    axes[1].set_ylabel('Petal Width (cm)')
    axes[1].set_title(f'Petal Coordinates\n({len(available_data)} available points)')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # STEP 4: Imputation Methods
    print(f"\nSTEP 4: Metode Imputation untuk Mengisi Missing Values")
    
    # Siapkan berbagai metode imputation
    imputation_methods = {}
    
    # Method 1: Mean Imputation
    df_mean_imputed = df_with_nan.copy()
    for feature in features:
        mean_value = df_mean_imputed[feature].mean()
        df_mean_imputed[feature].fillna(mean_value, inplace=True)
    imputation_methods['mean'] = df_mean_imputed
    
    # Method 2: Median Imputation
    df_median_imputed = df_with_nan.copy()
    for feature in features:
        median_value = df_median_imputed[feature].median()
        df_median_imputed[feature].fillna(median_value, inplace=True)
    imputation_methods['median'] = df_median_imputed
    
    # Method 3: Mode/Most Frequent per Species
    df_species_imputed = df_with_nan.copy()
    for feature in features:
        for species in [0, 1, 2]:
            species_mask = df_species_imputed['species'] == species
            species_data = df_species_imputed.loc[species_mask, feature]
            species_median = species_data.median()
            
            missing_mask = df_species_imputed[feature].isnull() & species_mask
            df_species_imputed.loc[missing_mask, feature] = species_median
    imputation_methods['species_median'] = df_species_imputed
    
    # Method 4: KNN Imputation (Simple version)
    from sklearn.impute import KNNImputer
    df_knn_imputed = df_with_nan.copy()
    
    # Hanya impute untuk features numerik
    knn_imputer = KNNImputer(n_neighbors=5)
    imputed_values = knn_imputer.fit_transform(df_knn_imputed[features])
    
    for i, feature in enumerate(features):
        df_knn_imputed[feature] = imputed_values[:, i]
    imputation_methods['knn'] = df_knn_imputed
    
    print(f"\nMetode Imputation yang digunakan:")
    print(f"   1. Mean Imputation: Menggunakan rata-rata kolom")
    print(f"   2. Median Imputation: Menggunakan median kolom")
    print(f"   3. Species Median: Menggunakan median per spesies")
    print(f"   4. KNN Imputation: Menggunakan 5 nearest neighbors")
    
    # STEP 5: Koordinat Perbandingan Metode Imputation
    print(f"\nSTEP 5: Koordinat Perbandingan Metode Imputation")
    
    # Show imputed coordinates untuk outlier indices
    print(f"\nKoordinat Imputed untuk Outlier Indices:")
    print(f"{'Method':<15} {'Index':<6} {'Sepal_L':<8} {'Sepal_W':<8} {'Petal_L':<8} {'Petal_W':<8}")
    print("-" * 70)
    
    # Show coordinates for first few outlier indices across methods
    sample_outlier_idx = outlier_indices[0] if outlier_indices else 0
    
    for method_name, df_imputed in list(imputation_methods.items())[:4]:
        method_display = {
            'mean': 'Mean',
            'median': 'Median', 
            'species_median': 'Species Med',
            'knn': 'KNN'
        }.get(method_name, method_name)
        
        row = df_imputed.loc[sample_outlier_idx]
        print(f"{method_display:<15} {sample_outlier_idx:<6} {row[features[0]]:<8.2f} {row[features[1]]:<8.2f} {row[features[2]]:<8.2f} {row[features[3]]:<8.2f}")
    
    # Visualisasi perbandingan koordinat
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Koordinat Perbandingan Metode Imputation', fontsize=16, fontweight='bold')
    
    colors = ['skyblue', 'lightcoral', 'lightpink', 'lightyellow']
    method_names = ['Mean', 'Median', 'Species Median', 'KNN']
    
    for i, (method, df_imputed) in enumerate(list(imputation_methods.items())[:4]):
        row = i // 2
        col = i % 2
        
        # Normal data (tidak berubah)
        normal_mask = ~df[features].index.isin(outlier_indices)
        normal_data = df_imputed[normal_mask]
        
        # Imputed data (yang tadinya outliers)
        imputed_data = df_imputed[df_imputed.index.isin(outlier_indices)]
        
        # Plot normal data
        axes[row, col].scatter(normal_data[features[0]], normal_data[features[1]], 
                             c=normal_data['species'], cmap='viridis', alpha=0.6, s=30, label='Normal')
        
        # Plot imputed outliers
        if len(imputed_data) > 0:
            axes[row, col].scatter(imputed_data[features[0]], imputed_data[features[1]], 
                                 color='red', s=80, marker='s', alpha=0.8, 
                                 label=f'Imputed ({len(imputed_data)})')
        
        axes[row, col].set_xlabel('Sepal Length (cm)')
        axes[row, col].set_ylabel('Sepal Width (cm)')
        axes[row, col].set_title(f'{method_names[i]} Imputation')
        axes[row, col].legend()
        axes[row, col].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # STEP 6: Evaluasi Metode Imputation
    print(f"\nSTEP 6: Pilih Metode Imputation Terbaik")
    
    # Pilih metode terbaik (species median untuk Iris dataset)
    recommended_method = 'species_median'
    df_processed = imputation_methods[recommended_method].copy()
    
    print(f"\n✓ Metode dipilih: SPECIES MEDIAN")
    print(f"   Alasan: Sesuai dengan karakteristik biologis per spesies")
    
    # Simple comparison table
    focus_feature = 'sepal length (cm)'
    print(f"\nPerbandingan hasil imputation ({focus_feature}):")
    print(f"{'Method':<12} {'Mean':<8}")
    print("-" * 25)
    
    # Show simple stats
    original_mean = df[focus_feature].mean()
    print(f"{'Original':<12} {original_mean:<7.2f}")
    
    for method, df_imputed in list(imputation_methods.items())[:3]:
        method_name = {'mean': 'Mean', 'median': 'Median', 'species_median': 'Species'}.get(method, method)
        imputed_mean = df_imputed[focus_feature].mean()
        print(f"{method_name:<12} {imputed_mean:<7.2f}")
    
    print(f"\n→ Species median dipilih karena mempertahankan perbedaan antar spesies")
    
    # STEP 7: Hasil Akhir Preprocessing
    print(f"\nSTEP 7: Hasil Akhir Preprocessing")
    
    # Tampilkan summary singkat
    print(f"\nPreprocessing Summary:")
    print(f"   • Outliers ditemukan: {outlier_count} data points")
    print(f"   • Metode imputation: Species-based median") 
    print(f"   • Dataset size tetap: 150 samples")
    
    # Koordinat comparison sederhana
    print(f"\nContoh perubahan koordinat outliers:")
    for idx in outlier_indices[:2]:  # Show first 2
        before = df.loc[idx, features[:2]]
        after = df_processed.loc[idx, features[:2]]
        print(f"   Index {idx}: {before.values} → {after.values}")
    
    # Visualisasi before vs after
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Before (dengan outliers)
    for species in df['species'].unique():
        species_data = df[df['species'] == species]
        ax1.scatter(species_data[features[0]], species_data[features[1]], 
                   label=species, alpha=0.7)
    # Mark outliers
    outliers_original = df.loc[outlier_indices]
    ax1.scatter(outliers_original[features[0]], outliers_original[features[1]], 
               color='red', marker='x', s=100, label='Outliers')
    ax1.set_title('Before: Data dengan Outliers')
    ax1.set_xlabel('Sepal Length (cm)')
    ax1.set_ylabel('Sepal Width (cm)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # After (imputed)
    for species in df_processed['species'].unique():
        species_data = df_processed[df_processed['species'] == species]
        ax2.scatter(species_data[features[0]], species_data[features[1]], 
                   label=species, alpha=0.7)
    # Mark imputed points
    imputed_points = df_processed.loc[outlier_indices]
    ax2.scatter(imputed_points[features[0]], imputed_points[features[1]], 
               color='green', marker='s', s=100, label='Imputed')
    ax2.set_title('After: Data dengan Imputation')
    ax2.set_xlabel('Sepal Length (cm)')
    ax2.set_ylabel('Sepal Width (cm)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✓ PREPROCESSING OUTLIER SELESAI")
    print(f"   Data siap untuk feature scaling dan modeling")
    
else:
    print("Tidak dapat melakukan outlier removal karena data atau hasil outlier tidak tersedia")
    df_processed = df.copy() if df is not None else None

=== OUTLIER REMOVAL DAN IMPUTATION PROCESS ===
Tidak dapat melakukan outlier removal karena data atau hasil outlier tidak tersedia


## 4. Feature Scaling dan Engineering

In [46]:
# Feature Scaling dan Engineering menggunakan PyCaret
print("=== FEATURE SCALING DAN ENGINEERING DENGAN PYCARET ===")

if df_processed is not None:
    
    # Analisis distribusi untuk scaling
    print(f"\nAnalisis distribusi fitur:")
    for feature in features[:2]:  # Show 2 features saja
        stats = df_processed[feature].describe()
        print(f"   • {feature.split('(')[0].strip()}: Mean={stats['mean']:.2f}, Std={stats['std']:.2f}")
    
    # Setup PyCaret preprocessing
    try:
        from pycaret.classification import setup as classification_setup, get_config
        
        print(f"\nSetting up PyCaret preprocessing...")
        
        # Setup PyCaret dengan feature scaling
        clf_env = classification_setup(
            data=df_processed[features + ['species']],
            target='species',
            session_id=42,
            train_size=0.8,
            normalize=True,
            normalize_method='zscore',
            feature_interaction=True,
            feature_ratio=True,
            polynomial_features=False,
            silent=True,
            verbose=False
        )
        
        # Get processed data
        X_train_pycaret = get_config('X_train')
        X_test_pycaret = get_config('X_test') 
        y_train_pycaret = get_config('y_train')
        y_test_pycaret = get_config('y_test')
        
        print(f"\nPyCaret preprocessing berhasil:")
        print(f"   • Features awal: {len(features)}")
        print(f"   • Features processed: {X_train_pycaret.shape[1]}")
        print(f"   • Training samples: {X_train_pycaret.shape[0]}")
        print(f"   • Test samples: {X_test_pycaret.shape[0]}")
        
        # Buat final datasets
        df_pycaret_processed = X_train_pycaret.copy()
        df_pycaret_processed['species'] = y_train_pycaret
        df_pycaret_test = X_test_pycaret.copy()
        df_pycaret_test['species'] = y_test_pycaret
        
        scaling_methods = {
            'pycaret': {
                'data': df_pycaret_processed,
                'test_data': df_pycaret_test,
                'scaler': 'PyCaret Pipeline',
                'description': 'PyCaret Z-score + Feature Engineering'
            }
        }
        
        recommended_method = 'pycaret'
        all_features = list(X_train_pycaret.columns)
        
    except Exception as e:
        print(f"PyCaret error: {e}")
        print("Menggunakan manual scaling...")
        
        # Manual scaling fallback
        from sklearn.preprocessing import StandardScaler, RobustScaler
        
        scaling_methods = {}
        
        # StandardScaler
        scaler_standard = StandardScaler()
        df_standard = df_processed.copy()
        df_standard[features] = scaler_standard.fit_transform(df_processed[features])
        scaling_methods['standard'] = {
            'data': df_standard,
            'scaler': scaler_standard,
            'description': 'StandardScaler (Z-score)'
        }
        
        # RobustScaler
        scaler_robust = RobustScaler()
        df_robust = df_processed.copy()
        df_robust[features] = scaler_robust.fit_transform(df_processed[features])
        scaling_methods['robust'] = {
            'data': df_robust,
            'scaler': scaler_robust,
            'description': 'RobustScaler (Robust to outliers)'
        }
        
        # Manual feature engineering
        for method_name, method_info in scaling_methods.items():
            df_current = method_info['data']
            
            # Simple feature engineering
            df_current['petal_area'] = df_current['petal length (cm)'] * df_current['petal width (cm)']
            df_current['sepal_area'] = df_current['sepal length (cm)'] * df_current['sepal width (cm)']
            df_current['petal_sepal_ratio'] = df_current['petal length (cm)'] / df_current['sepal length (cm)']
            
            scaling_methods[method_name]['data'] = df_current
        
        engineered_features = ['petal_area', 'sepal_area', 'petal_sepal_ratio']
        all_features = features + engineered_features
        
        print(f"\nManual scaling selesai:")
        print(f"   • Original features: {len(features)}")
        print(f"   • Engineered features: {len(engineered_features)}")
        print(f"   • Total features: {len(all_features)}")
        
        recommended_method = 'robust' if df_processed['strong_consensus'].sum() > 0 else 'standard'
    print(f"\nMETODE YANG DIPILIH: {recommended_method.upper()}")
    
    # Set final dataset
    if 'test_data' in scaling_methods[recommended_method]:
        # PyCaret case
        df_final = scaling_methods[recommended_method]['data']
        df_test_final = scaling_methods[recommended_method]['test_data']
        
        print(f"\nPyCaret Dataset Ready:")
        print(f"   • Training: {df_final.shape}")
        print(f"   • Test: {df_test_final.shape}")
        print(f"   • Features: {len(all_features)}")
        
    else:
        # Manual case
        df_final = scaling_methods[recommended_method]['data'].copy()
        
        print(f"\nManual Dataset Ready:")
        print(f"   • Total data: {df_final.shape}")
        print(f"   • Features: {len(all_features)}")
        print(f"   • Siap untuk train-test split")
    
    print(f"\n✓ PREPROCESSING SELESAI")
    print(f"   Dataset siap untuk modeling phase")

else:
    print("Tidak dapat melakukan feature scaling karena data processed tidak tersedia")
    df_final = None
    df_test_final = None
    final_scaler = None
    all_features = features if 'features' in locals() else []

=== FEATURE SCALING DAN ENGINEERING DENGAN PYCARET ===
Tidak dapat melakukan feature scaling karena data processed tidak tersedia


## 5. Train-Test Split dan Persiapan Final

In [47]:
# Train-Test Split dan Persiapan Dataset untuk Modeling dengan PyCaret
print("=== TRAIN-TEST SPLIT DAN PERSIAPAN FINAL DENGAN PYCARET ===")

if df_final is not None:
    
    # Check apakah sudah ada split dari PyCaret atau perlu manual split
    if df_test_final is not None:
        # PyCaret sudah melakukan split
        print(f"Menggunakan PyCaret train-test split:")
        print(f"   • PyCaret telah melakukan stratified split otomatis")
        print(f"   • Train set: {df_final.shape[0]} samples")
        print(f"   • Test set: {df_test_final.shape[0]} samples")
        
        # Extract features dan target dari PyCaret results
        target_col = 'species'
        feature_cols = [col for col in df_final.columns if col != target_col]
        
        X_train = df_final[feature_cols].copy()
        y_train = df_final[target_col].copy()
        X_test = df_test_final[feature_cols].copy()
        y_test = df_test_final[target_col].copy()
        
        all_features = feature_cols
        
        print(f"   • Features dari PyCaret: {len(all_features)}")
        
    else:
        # Manual split untuk non-PyCaret case
        print(f"Melakukan manual train-test split:")
        
        # Persiapan features dan target
        X = df_final[all_features].copy()
        y = df_final['species'].copy()
        
        print(f"Dataset Preparation:")
        print(f"   • Total samples: {len(df_final)}")
        print(f"   • Features: {len(all_features)}")
        print(f"   • Target classes: {y.nunique()} (setosa=0, versicolor=1, virginica=2)")
        
        # Train-test split dengan stratified sampling
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, 
            test_size=0.2, 
            random_state=42, 
            stratify=y
        )
        
        print(f"   • Train set: {len(X_train)} samples")
        print(f"   • Test set: {len(X_test)} samples")
    
    # Cek distribusi kelas
    print(f"\nDistribusi Kelas:")
    species_names = {0: 'setosa', 1: 'versicolor', 2: 'virginica'}
    
    # Train distribution
    train_dist = y_train.value_counts().sort_index()
    test_dist = y_test.value_counts().sort_index()
    
    print(f"{'Class':<12} {'Train Count':<12} {'Train %':<10} {'Test Count':<11} {'Test %':<8}")
    print("-" * 65)
    
    for class_idx in [0, 1, 2]:
        train_count = train_dist.get(class_idx, 0)
        test_count = test_dist.get(class_idx, 0)
        train_pct = (train_count / len(y_train)) * 100 if len(y_train) > 0 else 0
        test_pct = (test_count / len(y_test)) * 100 if len(y_test) > 0 else 0
        
        print(f"{species_names[class_idx]:<12} {train_count:<12} {train_pct:<9.1f}% {test_count:<11} {test_pct:<7.1f}%")
    
    # Informasi preprocessing untuk reproduksi
    preprocessing_method = 'pycaret' if df_test_final is not None else 'manual'
    
    preprocessing_info = {
        'outlier_treatment': 'consensus_based_removal',
        'scaling_method': preprocessing_method,
        'original_features': features,
        'processed_features': all_features,
        'total_features': len(all_features),
        'scaler': final_scaler,
        'train_size': len(X_train),
        'test_size': len(X_test),
        'random_state': 42,
        'preprocessing_pipeline': preprocessing_method
    }
    
    print(f"\nPreprocessing Information:")
    print(f"   • Outlier treatment: {preprocessing_info['outlier_treatment']}")
    print(f"   • Scaling method: {preprocessing_info['scaling_method']}")
    print(f"   • Total features: {preprocessing_info['total_features']}")
    print(f"   • Random state: {preprocessing_info['random_state']}")
    
    # Tampilkan sample dari dataset final
    print(f"\nSample Data Final (Train Set - First 5 rows):")
    if preprocessing_method == 'pycaret':
        # Show beberapa features penting dari PyCaret
        sample_features = list(X_train.columns)[:4]  # First 4 features dari PyCaret
    else:
        # Show original + engineered features
        sample_features = ['sepal length (cm)', 'petal length (cm)', 'petal_area', 'total_area']
    
    print(X_train[sample_features].head())
    
    print(f"\nTarget Labels (Train Set - First 10):")
    print(y_train.head(10).tolist())
    
    # PyCaret Data Preparation untuk modeling
    print(f"\nPERSIAPAN UNTUK PYCARET MODELING:")
    
    if preprocessing_method == 'pycaret':
        # Data sudah diproses oleh PyCaret
        df_pycaret = df_final.copy()
        df_pycaret_test = df_test_final.copy()
        
        print(f"   • Data telah diproses oleh PyCaret preprocessing pipeline")
        print(f"   • df_pycaret: {df_pycaret.shape} (training data)")
        print(f"   • df_pycaret_test: {df_pycaret_test.shape} (test data)")
        print(f"   • Target column: 'species'")
        print(f"   • Siap langsung untuk modeling")
        
    else:
        # Gabungkan X dan y untuk PyCaret
        df_pycaret = X_train.copy()
        df_pycaret['species'] = y_train
        
        # Siapkan test set terpisah untuk evaluasi final
        df_pycaret_test = X_test.copy()
        df_pycaret_test['species'] = y_test
        
        print(f"   • df_pycaret: {df_pycaret.shape} (untuk training & validation)")
        print(f"   • df_pycaret_test: {df_pycaret_test.shape} (untuk final evaluation)")
        print(f"   • Target column: 'species'")
        print(f"   • Feature columns: {len(all_features)} features")
    
    # Summary informasi untuk modeling
    print(f"\nSUMMARY PREPROCESSING:")
    print(f"   • Objective: Multi-class classification (3 classes)")
    print(f"   • Data quality: High (outliers handled dengan PyCaret)")
    print(f"   • Preprocessing: {preprocessing_method.upper()} pipeline")
    print(f"   • Features: {len(all_features)} total features")
    print(f"   • Class balance: Good (stratified split maintained)")
    print(f"   • Ready for: PyCaret automated ML pipeline")
    
    print(f"\nPREPROCESSING DENGAN PYCARET COMPLETED!")
    print(f"Dataset siap untuk modeling dengan PyCaret")
    
else:
    print("Tidak dapat melakukan train-test split karena dataset final tidak tersedia")
    X_train = X_test = y_train = y_test = None
    df_pycaret = df_pycaret_test = None
    preprocessing_info = None

=== TRAIN-TEST SPLIT DAN PERSIAPAN FINAL DENGAN PYCARET ===
Tidak dapat melakukan train-test split karena dataset final tidak tersedia


## 6. Summary dan Next Steps

In [48]:
# SUMMARY PREPROCESSING SEDERHANA
print("=" * 50)
print("        SUMMARY DATA PREPROCESSING")
print("=" * 50)

print("\nYANG SUDAH DILAKUKAN:")
print("   ✓ Load dataset Iris (150 data)")
print("   ✓ Deteksi outlier dengan 3 model (ABOD, KNN, COF)")
print("   ✓ Hapus outlier dan isi dengan imputation")
print("   ✓ Scaling dan feature engineering")
print("   ✓ Split data train-test")

if 'preprocessing_info' in locals() and preprocessing_info is not None:
    train_size = preprocessing_info.get('train_size', 'N/A')
    test_size = preprocessing_info.get('test_size', 'N/A')
    features_count = preprocessing_info.get('total_features', 'N/A')
    
    print(f"\nHASIL AKHIR:")
    print(f"   • Data train: {train_size} samples")
    print(f"   • Data test: {test_size} samples") 
    print(f"   • Total fitur: {features_count}")
    print(f"   • Kelas: 3 (setosa, versicolor, virginica)")

print(f"\nLANGKAH SELANJUTNYA:")
print(f"   1. Modeling dengan PyCaret")
print(f"   2. Evaluasi performa model")
print(f"   3. Pilih model terbaik")

if 'df_pycaret' in locals() and df_pycaret is not None:
    print(f"\n✓ Data siap untuk modeling!")
else:
    print(f"\n⚠ Jalankan semua cell preprocessing dulu")

print(f"\n" + "=" * 50)

        SUMMARY DATA PREPROCESSING

YANG SUDAH DILAKUKAN:
   ✓ Load dataset Iris (150 data)
   ✓ Deteksi outlier dengan 3 model (ABOD, KNN, COF)
   ✓ Hapus outlier dan isi dengan imputation
   ✓ Scaling dan feature engineering
   ✓ Split data train-test

LANGKAH SELANJUTNYA:
   1. Modeling dengan PyCaret
   2. Evaluasi performa model
   3. Pilih model terbaik

⚠ Jalankan semua cell preprocessing dulu

